<a href="https://colab.research.google.com/github/spenceleyyy/SEOC/blob/main/test_people_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Step 1: Install & Update YOLOv8 ---
!pip uninstall -y ultralytics
!pip install ultralytics --no-cache-dir

Found existing installation: ultralytics 8.3.90
Uninstalling ultralytics-8.3.90:
  Successfully uninstalled ultralytics-8.3.90
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.1/949.1 kB 20.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- Step 2: Fix Configuration Files ---
!wget -O /usr/local/lib/python3.11/dist-packages/ultralytics/cfg/default.yaml \
    https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/default.yaml

/usr/local/lib/python3.11/dist-packages/ultralytics/cfg/default.yaml: No such file or directory


In [ ]:
import os

# Define the directory path
tracker_dir = "/usr/local/lib/python3.11/dist-packages/ultralytics/cfg/track/"

# Check if the directory exists, and create it if not
if not os.path.exists(tracker_dir):
    os.makedirs(tracker_dir)  # Create the missing directories
    print(f"Created missing directory: {tracker_dir}")

# Now write the custom tracker config file
tracker_path = os.path.join(tracker_dir, "botsort.yaml")
botsort_config = """
low_conf: 0.1               # Ignore very low-confidence detections
new_track_thresh: 0.9       # Higher threshold for new tracks (avoids ID switching)
match_thresh: 0.99          # Stricter matching to keep IDs consistent
track_buffer: 200           # Keeps IDs alive longer during occlusions
reid: True                  # Enables Re-Identification (matches IDs after occlusions)
"""

with open(tracker_path, "w") as f:
    f.write(botsort_config)

print(f"Custom tracker file created at {tracker_path}")

Created missing directory: /usr/local/lib/python3.11/dist-packages/ultralytics/cfg/track/
Custom tracker file created at /usr/local/lib/python3.11/dist-packages/ultralytics/cfg/track/botsort.yaml


In [ ]:
# --- Step 4: Run YOLOv8 Tracking ---
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Run tracking
results = model.track(source="/content/drive/My Drive/sEOC/test.MOV",
                      save=True,
                      persist=True,
                      tracker=tracker_path,  # Ensure correct tracker path  # Keep streaming mode enabled
                      device='cpu',
                      vid_stride=5)

# Convert generator to list (if tracking works)
if isinstance(results, list):
    results_list = list(results)
    for r in results_list:
        print(r)
else:
    print("⚠️ Tracking did not return a list, check output type:", type(results))

ImportError: cannot import name 'YOLO' from 'ultralytics' (unknown location)

In [ ]:
# --- Step 5: Verify Output ---
if os.path.exists("runs/detect/track/"):
    print("Results folder found! Listing files...")
    print(os.listdir("runs/detect/track/"))
else:
    print("ERROR: Tracking output folder not found!")

In [ ]:
# --- Step 6: Download the Processed Video ---
from google.colab import files

processed_video = "tracked_output.mp4"  # Change this if needed
video_path = f"runs/detect/track9/{processed_video}"

if os.path.exists(video_path):
    print("Downloading processed video...")
    files.download(video_path)
else:
    print("ERROR: Processed video not found!")


In [ ]:
# --- Step 7: (Optional) Download Tracking Data ---
tracking_data_path = "runs/detect/track9/tracking_data.csv"
if os.path.exists(tracking_data_path):
    print("Downloading tracking data...")
    files.download(tracking_data_path)
else:
    print("No tracking data found!")

**Important:** After installing or updating packages like `ultralytics`, it's often necessary to restart your Colab runtime to ensure that the changes take effect. You can do this by going to the menu `Runtime` > `Restart runtime`.

Once the runtime has restarted, please run all the cells in the notebook again, starting from the first cell.